In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Project #1. Flows on the London Subway ##

For this project we will use data on the London subway. The data is organized as follows:
- file ''connections'' contains data on the direct links between two stations; for instance, we can go from Station 11 to Station 163 (and vice versa) by taking line 1; it will take us 1 minute;
- file ''stations'' contains information on identifiers of each station;
- file ''stations_restricted'' contains a subset of the stations from stations.csv

From this information, we can construct a network as follows:
- the nodes are the stations;
- there is an arc between two nodes if there is a direct link between the two corresponding stations in the file ''connections'' (assume every connection corresponds to two arcs in opposite directions); for instance, there is an arc (11,163), and arc (163,11), but no arc (1,170);
- define capacities as follows:
1.   Generate a random number between 5 and 10 for each line: this is the capacity of the line.
2.   The capacity of an arc between two stations has the same capacity as the sum of the capacities of the lines that link them. For instance, the stations of King's Cross St. Pancras (#145) and Euston Square (#90) are linked by lines 3,6,8. Let's say we randomly sampled the capacity of 6,8,4 for lines 3,6,8. The capacity of the arc (145,90) is 6+8+4=18. Similarly, the capacity of the arc (90,145) is also 6+8+4=18.

The project is composed of 4 questions, all to be solved using Linear Program and to be completed in this order:

1. We want to compute how many passengers can transit on average from the north-east part of the city center, to the south-west. To do so, compute a maximum flow between King's Cross St. Pancras (#145) and Hammersmith (#110) on the instance in the file ''connections'' (ignore the time it takes to go between stations).
2. We want to understand how many people can be evacuated from the city center within 18 minutes. To do so, restrict to the stations in the file ''stations_restricted'' (and the corresponding arcs). Compute the maximum flow over time between King's Cross St. Pancras (#145) and Victoria (#273) for T=18.
3. Consider again the restriction to the stations in the file ''stations_restricted''. For security reasons, in the stations Oxford Circus (#192) and Embankment (#87), at any given time there can be at most 20 passengers in transit. Solve problem 2) above under this additional restriction.
4. **4) Will not be evaluated unless parts 1),2),3) have been correctly solved** Choose an improvement to the line and write a linear program that finds the *best* way to realize your improvement. For instance, you could assume you have a budget of 10 units of additional capacity to be added to the network, and you want to decide where to add them so that the maximum flow in part 1) above increases the most (it is ok to have an improvement that requires an integer solution, but the solution you get is fractional).





In [5]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

np.random.seed(43)

connections_df = pd.read_csv('connections.csv')

connections_df['capacity'] = connections_df.groupby('line')['line'].transform(lambda x: np.random.randint(5, 11))

connections_df['station_pair'] = connections_df.apply(lambda row: tuple(sorted((row['station1'], row['station2']))), axis=1)
aggregated_capacity_df = connections_df.groupby('station_pair')['capacity'].sum().reset_index()
aggregated_capacity_df[['station1', 'station2']] = pd.DataFrame(aggregated_capacity_df['station_pair'].tolist(), index=aggregated_capacity_df.index)
aggregated_capacity_df = aggregated_capacity_df[['station1', 'station2', 'capacity']]

model = gp.Model("max_flow")

flow_vars = {}

edges = []
for _, row in aggregated_capacity_df.iterrows():
    station1 = row['station1']
    station2 = row['station2']
    capacity = row['capacity'] 

    flow_vars[(station1, station2)] = model.addVar(lb=0, ub=capacity, vtype=GRB.INTEGER, name=f'flow_{station1}_{station2}')
    flow_vars[(station2, station1)] = model.addVar(lb=0, ub=capacity, vtype=GRB.INTEGER, name=f'flow_{station2}_{station1}')

    edges.append((station1, station2, capacity))
    edges.append((station2, station1, capacity))

    model.addConstr(flow_vars[(station1, station2)] <= capacity, name=f'cap_consistency_{station1}_{station2}')
    model.addConstr(flow_vars[(station2, station1)] <= capacity, name=f'cap_consistency_{station2}_{station1}')

source = 145  # King's Cross St. Pancras
target = 110  # Hammersmith

model.setObjective(gp.quicksum(flow_vars[(i, target)] for (i, j, _) in edges if j == target), GRB.MAXIMIZE)

stations = set(aggregated_capacity_df['station1']).union(set(aggregated_capacity_df['station2']))

for station in stations:
    if station != source and station != target:
        inflow = gp.quicksum(flow_vars[(i, station)] for (i, j, _) in edges if j == station)
        outflow = gp.quicksum(flow_vars[(station, j)] for (i, j, _) in edges if i == station)
        model.addConstr(inflow == outflow, name=f'flow_conservation_{station}')

model.optimize()

if model.status == GRB.OPTIMAL:
    print(f"Maximum flow from King's Cross St. Pancras (#145) to Hammersmith (#110): {model.objVal}")
else:
    print("No optimal solution found.")


model_improved = gp.Model("max_flow_with_capacity_increase")

flow_vars_improved = {}
delta_capacity_vars = {}
for _, row in aggregated_capacity_df.iterrows():
    station1 = int(row['station1'])
    station2 = int(row['station2'])
    capacity = row['capacity']

    flow_vars_improved[(station1, station2)] = model_improved.addVar(lb=0, vtype=GRB.INTEGER,
                                                                     name=f'flow_{station1}_{station2}')
    flow_vars_improved[(station2, station1)] = model_improved.addVar(lb=0, vtype=GRB.INTEGER,
                                                                     name=f'flow_{station2}_{station1}')
    delta_capacity_vars[(station1, station2)] = model_improved.addVar(lb=0, ub=10, vtype=GRB.INTEGER,
                                                                      name=f'delta_c_{station1}_{station2}')
    delta_capacity_vars[(station2, station1)] = model_improved.addVar(lb=0, ub=10, vtype=GRB.INTEGER,
                                                                      name=f'delta_c_{station2}_{station1}')

    model_improved.addConstr(
        flow_vars_improved[(station1, station2)] <= capacity + delta_capacity_vars[(station1, station2)],
        name=f'capacity_constraint_{station1}_{station2}'
    )
    model_improved.addConstr(
        flow_vars_improved[(station2, station1)] <= capacity + delta_capacity_vars[(station2, station1)],
        name=f'capacity_constraint_{station2}_{station1}'
    )

total_additional_capacity = gp.quicksum(delta_capacity_vars.values())
model_improved.addConstr(total_additional_capacity <= 10, name='budget_constraint')

for station in stations:
    if station == source:
        continue
    elif station == target:
        continue
    else:
        inflow = gp.quicksum(flow_vars_improved[(i, station)] for (i, j, _) in edges if j == station)
        outflow = gp.quicksum(flow_vars_improved[(station, j)] for (i, j, _) in edges if i == station)
        model_improved.addConstr(inflow == outflow, name=f'flow_conservation_{station}')

total_outflow_source_improved = gp.quicksum(flow_vars_improved[(source, j)] for (i, j, _) in edges if i == source)
total_inflow_sink_improved = gp.quicksum(flow_vars_improved[(i, target)] for (i, j, _) in edges if j == target)
model_improved.addConstr(total_outflow_source_improved == total_inflow_sink_improved, name='source_sink_flow_balance_improved')

model_improved.setObjective(total_outflow_source_improved, GRB.MAXIMIZE)

model_improved.optimize()

if model_improved.status == GRB.OPTIMAL:
    improved_max_flow = model_improved.objVal
    print(f"\nThe maximum flow after increasing capacity from King's Cross St. Pancras (#145) to Hammersmith (#110).：{improved_max_flow}")
    print(f"Total capacity increase.：{total_additional_capacity.getValue()}")

    print("\nCapacity Increase Allocation:")
    for (i, j) in delta_capacity_vars:
        delta = delta_capacity_vars[(i, j)].X
        if delta > 0:
            print(f"The capacity of edge ({i}, {j}) has increased by {delta}.")
else:
    print("No optimal solution found.")



Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 7 155U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 1072 rows, 752 columns and 2234 nonzeros
Model fingerprint: 0x3cffe92f
Variable types: 0 continuous, 752 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+00, 2e+01]
  RHS range        [5e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 41.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 14 available processors)

Solution count 2: 41 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.100000000000e+01, best bound 4.100000000000e+01, gap 0.0000%
Maximum flow from King's Cross St. Pancras (#145) to Hammersmith (#110): 41.0
Gurobi Optim